# Spotify mainstreamness analyzer

##### How much mainstream is your music?

![Image](pipeline.png)

## Spotify

Mette a disposizione una api, per ottenere informazioni su brani, artisti, playlist e altro.
In particolare mette a disposizione una serie di informazioni su ogni brano, di due tipi:
Basso livello: dove fa un'analisi molto dettagliata su ogni secondo del brano, con pitch, timbro, beats e altro.
Alto livello: una serie di indici per classificare il brano, che forniscono informazioni più esplicite e immediate. Ci sono 7 indici che definiscono caratteristiche di un brano, ognuno può assumere valori da 0 a 1 compresi:
- __speechiness__:
    quanto parlato c'è nel brano
- __danceability__:
    quanto è ballabile
- __liveness__:
    quanto il suono è dal vivo (come concerti)
- __valence__:
    più o meno misura la "felicità" del brano
- __energy__:
    misura intensità e attività
- __instrumentalness__:
    quanto è strumentale
- __acousticness__:
    quanto è appunto acustica (basso: elettronica, alto: classica)

## Python

Per eseguire richieste all'api di spotify ho usato una libreria apposita di python: _spotipy_.
Prima di poter fare richieste però è necessario prima però creare un'applicazione sulla dashboard di spotify developer, e connettere lo script ad essa con _client id_ e _secret_.
Successivamente si possono effettuare richieste del tipo:<br>
`sp.search("Timber", limit=4)` Restituisce un json con i primi 4 risultati della ricerca "Timber"<br>
`sp.audio_features([track_id])` Restituisce un oggetto json con quegli indici di features relativi all'id del brano inserito

Quindi costruisco un array di oggetti json del tipo:
```
{
    "name": "nome brano",
    "playlist": "nome playlist di appartenenza (se presente)"
    "track_id": "id di spotify del brano",
    "speechiness": valore,
    "energy": valore,
    ..
}
```
Per mandarlo a Logstash con tcp su porta 6000

## Logstash

Nella sua configurazione definisco come input: tcp con porta 6000, e come output: stdout e kafka sul topic: topic_1

## Kafka

Rende disponibili i dati sulla porta 9092 e topic topic_1

## Spark

Usando pyspark, la libreria per spark di python. Creo un input structured stream da kafka, e organizzo i messaggi in un dataframe.
Creo 2 dataframe di training, nel primo (_train1_ ) carico il dataset csv dalle top canzoni su spotify della settimana e della giornata, per un totale di 150 brani.
Il secondo dataframe di training (train_inverse), invece lo creo basandomi sul primo, ad ogni valore mappo 1 - valore in _train1_ in modo da avere un set fittizio di canzoni con valori esattamente opposti a quelle più popolari.
Quindi aggiungo a entrambi una colonna impostata a 1 per tutto _train1_ e a 0 per tutto train_inverse

Per ogni brano o insieme di brani in arrivo, eseguo le operazioni in pipeline, che aggiungeranno la colonna di _prediction_, che avrà un valore in base a quanto gli indici di speechness, valence, ecc.. saranno simili a quelli delle top songs.

## Per avviare

Dalla directory principale del progetto, eseguire i seguenti comandi:

docker-compose up -d

###### Per avviare la cli per eseguire le query su spotify:
 
./sproducer.sh